In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import seaborn as sns
from datetime import datetime, date, timedelta
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [3]:
df = pd.read_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\Coupon_data2019.xlsx')
df.head()


,Ticket Number,Issue date,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,SMS Total Org,GDS Total Org,Org Issuer Company,Org Distribution Channel
0,2482100288006,2018-06-05,Kirsten,Lauritzen,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,W,599,0,homepage,6I
1,2482100288006,2018-06-05,Kirsten,Lauritzen,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,H,699,0,homepage,6I
2,2482100288007,2018-06-05,Gerhard,Nissen,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,W,599,0,homepage,6I
3,2482100288007,2018-06-05,Gerhard,Nissen,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,H,699,0,homepage,6I
4,2482100288010,2018-06-05,Michelle,Lauritzen Frøhlich,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,O,349,0,homepage,6I


In [6]:
df=df.rename(columns = {'Issue date':'Booking creation time'})
#df['Booking creation time']=pd.to_datetime(df['Booking creation time']) #from string to date

In [7]:
df.shape

(72761, 16)

In [8]:
#Focusing only on Sonderborg-Copenhagen or Copenhagen-Sonderborg 

options = ['SGD', 'CPH'] 
    
# selecting rows based on condition 
df = df[df['Origin'].isin(options)] 
df = df[df['Destination'].isin(options)] 

In [9]:
df.shape

(64706, 16)

In [10]:
df['a']=df['Fare Basis Code'].copy()

In [11]:
b=df['a'].str[1:4]
b

0           
1           
2           
3           
4           
        ... 
72756    VA2
72757    SEM
72758    SEM
72759    SEM
72760    SEM
Name: a, Length: 64706, dtype: object

In [12]:
df['Fare Basis Code']=b
df=df.rename(columns = {'a':'Fare Basis Code not cut'})
df.head()
df['Full Name']=df["PAX first name"]+" "+df["PAX last name"]
df['Full Name']=df['Full Name'].str.lower()

In [152]:
print(df.shape)
df.head()

(35827, 19)


,Ticket Number,Booking creation time,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,SMS Total Org,GDS Total Org,Org Issuer Company,Org Distribution Channel,Contact person mobile,Fare Basis Code not cut,Full Name
140,2.482100e+12,2018-10-05,Michael Tamstrup,Jensen,Adult,SGD,CPH,2019-08-24,07:45,2019-08-24,08:30,BAS,199.0,0.0,Ecommerce,6I,+4561209587,TBASRT,michael tamstrup jensen
141,2.482100e+12,2018-10-05,Michael Tamstrup,Jensen,Adult,CPH,SGD,2019-09-09,11:15,2019-09-09,12:00,BAS,199.0,0.0,Ecommerce,6I,+4561209587,TBASRT,michael tamstrup jensen
142,2.482100e+12,2018-10-05,Mette Viskum,Jensen,Adult,SGD,CPH,2019-08-24,07:45,2019-08-24,08:30,BAS,199.0,0.0,Ecommerce,6I,+4561209587,TBASRT,mette viskum jensen
143,2.482100e+12,2018-10-05,Mette Viskum,Jensen,Adult,CPH,SGD,2019-09-09,11:15,2019-09-09,12:00,BAS,199.0,0.0,Ecommerce,6I,+4561209587,TBASRT,mette viskum jensen
144,2.482100e+12,2018-10-05,Thomas Viskum,Jensen,Adult,SGD,CPH,2019-08-24,07:45,2019-08-24,08:30,BAS,199.0,0.0,Ecommerce,6I,+4561209587,TBASRT,thomas viskum jensen


In [13]:
dfr = pd.read_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\RESERVATION_REPORT2019.xlsx')
dfr.head()


,eTicket number,Passenger first name,Passenger last name,Passenger mobile,Passenger email,Contact person email,Unnamed: 6,Booking creation time
0,2.482100e+12,Carmen Krull,Madsen,NaN,NaN,mc@juca.nu,NaN,2018-12-28 19:08:14
1,2.482100e+12,Mikkel,Friis-Thomsen,+4529123664,NaN,Mft@pfa.dk,NaN,2019-09-02 13:52:17
2,2.483122e+12,BENNY,ENGELBRECHT,NaN,NaN,NaN,NaN,2019-01-01 12:01:48
3,2.482100e+12,Kia Fiona,Rejmers,NaN,NaN,ore@dw.dk,NaN,2018-08-30 16:42:38
4,2.482100e+12,Lena,Kristiansen,+4525721546,NaN,lena0701@hotmail.com,NaN,2019-04-22 08:13:26


In [14]:
dfr=dfr.rename(columns = {'eTicket number':'Ticket Number'})

In [15]:
dfr['Booking creation time']=pd.to_datetime(dfr['Booking creation time']) #from string to date
dfr['Booking Hour']=pd.to_datetime(dfr['Booking creation time']).dt.hour
dfr['Booking Weekday']=pd.to_datetime(dfr['Booking creation time']).dt.weekday

In [16]:
dfr['Full Name']=dfr["Passenger first name"]+" "+dfr["Passenger last name"]
dfr['Full Name']=dfr['Full Name'].str.lower()
dfr.head()


,Ticket Number,Passenger first name,Passenger last name,Passenger mobile,Passenger email,Contact person email,Unnamed: 6,Booking creation time,Booking Hour,Booking Weekday,Full Name
0,2.482100e+12,Carmen Krull,Madsen,NaN,NaN,mc@juca.nu,NaN,2018-12-28 19:08:14,19,4,carmen krull madsen
1,2.482100e+12,Mikkel,Friis-Thomsen,+4529123664,NaN,Mft@pfa.dk,NaN,2019-09-02 13:52:17,13,0,mikkel friis-thomsen
2,2.483122e+12,BENNY,ENGELBRECHT,NaN,NaN,NaN,NaN,2019-01-01 12:01:48,12,1,benny engelbrecht
3,2.482100e+12,Kia Fiona,Rejmers,NaN,NaN,ore@dw.dk,NaN,2018-08-30 16:42:38,16,3,kia fiona rejmers
4,2.482100e+12,Lena,Kristiansen,+4525721546,NaN,lena0701@hotmail.com,NaN,2019-04-22 08:13:26,8,0,lena kristiansen


In [17]:
dfr2=dfr.copy()
dfr2.head
#domains=['@hotmail','@gmail','@yahoo', '@aol', '@msn', '@wanadoo', '@orange', '@comcast', '@live', '@gmx']
dfr2['Contact person email']=dfr2['Contact person email'].str.contains('@hotmail|@gmail|@yahoo|@aol|@msn|@wanadoo|@orange|@comcast|@live|@gmx')
dfr2['Passenger email']=dfr2['Passenger email'].str.contains('@hotmail|@gmail|@yahoo|@aol|@msn|@wanadoo|@orange|@comcast|@live|@gmx')


In [18]:
dfr['Type']=dfr2['Contact person email']
dfr['Type'] = dfr['Type'].map({True: 'Leisure', False: 'Business'}) 

In [20]:
dfr['Booking creation day']=pd.to_datetime(dfr['Booking creation time']).dt.date

In [21]:
dfr.head()

,Ticket Number,Passenger first name,Passenger last name,Passenger mobile,Passenger email,Contact person email,Unnamed: 6,Booking creation time,Booking Hour,Booking Weekday,Full Name,Type,Booking creation day
0,2.482100e+12,Carmen Krull,Madsen,NaN,NaN,mc@juca.nu,NaN,2018-12-28 19:08:14,19,4,carmen krull madsen,Business,2018-12-28
1,2.482100e+12,Mikkel,Friis-Thomsen,+4529123664,NaN,Mft@pfa.dk,NaN,2019-09-02 13:52:17,13,0,mikkel friis-thomsen,Business,2019-09-02
2,2.483122e+12,BENNY,ENGELBRECHT,NaN,NaN,NaN,NaN,2019-01-01 12:01:48,12,1,benny engelbrecht,NaN,2019-01-01
3,2.482100e+12,Kia Fiona,Rejmers,NaN,NaN,ore@dw.dk,NaN,2018-08-30 16:42:38,16,3,kia fiona rejmers,Business,2018-08-30
4,2.482100e+12,Lena,Kristiansen,+4525721546,NaN,lena0701@hotmail.com,NaN,2019-04-22 08:13:26,8,0,lena kristiansen,Leisure,2019-04-22


In [24]:
df['Ticket Number'] = df['Ticket Number'].astype('float64', copy=False)
df.dtypes
df.head()

,Ticket Number,Booking creation time,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,SMS Total Org,GDS Total Org,Org Issuer Company,Org Distribution Channel,Fare Basis Code not cut,Full Name
0,2.482100e+12,2018-06-05,Kirsten,Lauritzen,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,599,0,homepage,6I,W,kirsten lauritzen
1,2.482100e+12,2018-06-05,Kirsten,Lauritzen,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,,699,0,homepage,6I,H,kirsten lauritzen
2,2.482100e+12,2018-06-05,Gerhard,Nissen,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,599,0,homepage,6I,W,gerhard nissen
3,2.482100e+12,2018-06-05,Gerhard,Nissen,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,,699,0,homepage,6I,H,gerhard nissen
4,2.482100e+12,2018-06-05,Michelle,Lauritzen Frøhlich,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,349,0,homepage,6I,O,michelle lauritzen frøhlich


In [29]:
merged_df = pd.merge(df,dfr, how = 'inner', on = ['Ticket Number', 'Full Name'])

In [30]:
merged_df.shape

(98646, 29)

In [31]:
del merged_df['Passenger first name'],merged_df['Unnamed: 6'], merged_df['Passenger last name'], merged_df['PAX first name'], merged_df['PAX last name'], 

#solution created doubled duplicated rows 
merged_df=merged_df.drop_duplicates()
merged_df.head()

,Ticket Number,Booking creation time_x,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,...,Fare Basis Code not cut,Full Name,Passenger mobile,Passenger email,Contact person email,Booking creation time_y,Booking Hour,Booking Weekday,Type,Booking creation day
0,2.482100e+12,2018-06-05,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,...,W,kirsten lauritzen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05
2,2.482100e+12,2018-06-05,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,,...,H,kirsten lauritzen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05
4,2.482100e+12,2018-06-05,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,...,W,gerhard nissen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05
6,2.482100e+12,2018-06-05,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,,...,H,gerhard nissen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05
8,2.482100e+12,2018-06-05,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,...,O,michelle lauritzen frøhlich,NaN,NaN,Michelle-12@live.dk,2018-06-05 16:54:18,16,1,Leisure,2018-06-05


In [33]:
#merged_df['Departure Date']=pd.to_datetime(merged_df['Departure Date'])
#merged_df['Days before']=merged_df['Departure Date']-merged_df["Booking creation time_x"]
merged_df["Sum of cost"]=merged_df['SMS Total Org']+merged_df['GDS Total Org']
merged_df = merged_df.dropna(subset=["Sum of cost"])
print(merged_df.shape)


(57840, 25)


In [34]:
merged_df.head()

,Ticket Number,Booking creation time_x,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,...,Full Name,Passenger mobile,Passenger email,Contact person email,Booking creation time_y,Booking Hour,Booking Weekday,Type,Booking creation day,Sum of cost
0,2.482100e+12,2018-06-05,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,...,kirsten lauritzen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05,599
2,2.482100e+12,2018-06-05,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,,...,kirsten lauritzen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05,699
4,2.482100e+12,2018-06-05,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,...,gerhard nissen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05,599
6,2.482100e+12,2018-06-05,Adult,CPH,SGD,2019-02-19,08:15,2019-02-19,09:00,,...,gerhard nissen,NaN,NaN,Kirstenlauritzen@live.dk,2018-06-05 16:47:24,16,1,Leisure,2018-06-05,699
8,2.482100e+12,2018-06-05,Adult,SGD,CPH,2019-02-04,09:30,2019-02-04,10:15,,...,michelle lauritzen frøhlich,NaN,NaN,Michelle-12@live.dk,2018-06-05 16:54:18,16,1,Leisure,2018-06-05,349


In [35]:
merged_df.to_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\READY DATASETS\merged2019.xlsx', index=False)